<a href="https://colab.research.google.com/github/FarhanSajid1/MachineLearning/blob/master/Stock_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

from google.colab import files
import os
uploaded = files.upload()



Saving twitter.csv to twitter.csv


In [0]:
df = pd.read_csv('twitter.csv')
print(len(df))
print(df.head())
print(df.info())

5000
            created_at                                               text  \
0  2013-01-01 20:10:04  Kickers on my watchlist $XIDE $TRIT $SOQ $PNK ...   
1  2013-01-01 20:33:37  "@user: $AAPL MOVIE. 55% return for the FEAR/G...   
2  2013-01-01 21:43:41  @user I'd be afraid to short $AMZN - they are ...   
3  2013-01-02 01:49:48                             $MNTA Over $12.00 URL    
4  2013-01-02 01:51:33                              $OI  Over $21.37 URL    

  sentiment  
0  positive  
1  positive  
2  positive  
3  positive  
4  positive  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
created_at    5000 non-null object
text          5000 non-null object
sentiment     5000 non-null object
dtypes: object(3)
memory usage: 117.3+ KB
None


In [0]:
print(len(df.loc[df['sentiment'] == 'positive'])) #this returns the amount of positive samples we have
print(len(df.loc[df['sentiment'] == 'negative'])) #this returns the amount of negative samples that we havve
print(3350 + 1650)

3350
1650
5000


In [0]:
#Create numpy arrays with the values that we need. 
text = df['text'].values
labels = df['sentiment'].values
print(text.dtype)

object


In [0]:
# preprocess the data so that we can read it in the RNN

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
#figuring out how many words should be in each padding sequence
len_tokens=[len(token) for token in text]
mean_value = np.mean(len_tokens) # 76.40 is the mean
print(np.max(len_tokens)) # 140
print(mean_value) # 76.40



Using TensorFlow backend.


140
76.4056


In [0]:
# creating testing and validation sampling
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
max_len = int(mean_value + 1* np.std(len_tokens)) # 112 l
training = int(len((text)) * .8) # 4000
testing = int(len((text)) * .2) # 1000

max_words = 10000 # maximum words in our universal vocabulary
tokenizer = Tokenizer(num_words = max_words)
#fit the tokenizer on the text
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)

#padding the data
data = pad_sequences(sequences, maxlen = max_len)

labels = np.asarray(labels)


#shuffling and splitting the data!
indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels = labels[indices]

X_train = data[: training]
X_test = data[training:]

y_train = labels[:training]
y_test = labels[training:]

#we need to label binarize the data

encoder = LabelBinarizer()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape) #this is for the sigmoid output binary
print(y_test.shape)

(4000, 112)
(1000, 112)
(4000, 1)
(1000, 1)


In [0]:
print(data[0]) #training sequence after the padding

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0   12 1167  190   28 4327  108   81   32 1130
 4328   79    4  589  409    2  144   10   43   46  547  167   51   38]


In [0]:
print(text[0])

Kickers on my watchlist $XIDE $TRIT $SOQ $PNK $CPWR $BPZ $ALJ  trade method 1 or method 2, see prev posts


In [0]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
reverse_word_map

In [0]:
from keras.models import Sequential
from keras import regularizers
from keras import layers
from keras.layers import Bidirectional
from keras.layers import LSTM, GRU


model = Sequential()
model.add(layers.Embedding(max_words, 128))
# model.add(layers.(256,return_sequences=True, recurrent_dropout = .1, kernel_regularizer = regularizers.l2(0.001)))
# model.add(layers.Dropout(.1))
# model.add(layers.GRU(128,return_sequences=True, recurrent_dropout = .2, kernel_regularizer = regularizers.l2(0.001)))
# model.add(layers.Dropout(.2))
# model.add(layers.GRU(64,return_sequences=True, recurrent_dropout = .1, kernel_regularizer = regularizers.l2(0.001)))
# model.add(layers.Dropout(.2))
# model.add(layers.GRU(32, kernel_regularizer = regularizers.l2(0.001)))
# model.add(layers.Dense(1,activation='sigmoid'))

model.add(Bidirectional(GRU(128, return_sequences = True,  kernel_regularizer = regularizers.l2(0.001), recurrent_dropout = .2)))
model.add(layers.Dropout(.2))
model.add(Bidirectional(GRU(64, return_sequences = True,  kernel_regularizer = regularizers.l2(0.1))))
model.add(Bidirectional(GRU(32,  kernel_regularizer = regularizers.l2(0.1))))
model.add(layers.Dense(1, activation='sigmoid'))


model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])
history = model.fit(X_train, y_train, epochs = 8, batch_size = 128, validation_data = [X_test, y_test])

Train on 4000 samples, validate on 1000 samples
Epoch 1/8
4000/4000 [==============================] - 76s 19ms/step - loss: 40.3196 - acc: 0.6680 - val_loss: 23.9190 - val_acc: 0.7040
Epoch 2/8
4000/4000 [==============================] - 71s 18ms/step - loss: 15.6305 - acc: 0.7935 - val_loss: 8.6860 - val_acc: 0.7720
Epoch 3/8
4000/4000 [==============================] - 71s 18ms/step - loss: 5.0835 - acc: 0.8708 - val_loss: 2.5653 - val_acc: 0.7520
Epoch 4/8
4000/4000 [==============================] - 71s 18ms/step - loss: 1.3245 - acc: 0.9125 - val_loss: 1.0160 - val_acc: 0.7840
Epoch 5/8
4000/4000 [==============================] - 71s 18ms/step - loss: 0.3938 - acc: 0.9360 - val_loss: 1.0282 - val_acc: 0.7520
Epoch 6/8
4000/4000 [==============================] - 71s 18ms/step - loss: 0.2305 - acc: 0.9483 - val_loss: 0.6351 - val_acc: 0.7710
Epoch 7/8
4000/4000 [==============================] - 71s 18ms/step - loss: 0.1925 - acc: 0.9560 - val_loss: 0.5396 - val_acc: 0.7920
Epoc

In [0]:
#evaluating the accuracy of the network

result = model.evaluate(X_test, y_test)
print(result)

1000/1000 [==============================] - 6s 6ms/step
[0.848092288017273, 0.751]


In [0]:
# analyzing the misclassified text

y_predict = model.predict(x=X_test[0:100]) #testing on the first 100
y_predict = y_predict
y_predict.shape

(100, 1)

In [0]:
clas_pred = np.array([1 if p >.5 else 0 for p in y_predict])
# this is a one liner which will convert anything that is greater than .5 to a 1, making it fire a 1, and anything lower as a 0
#this way it makes classification easier. As opposed to numbers between 0-1 we have values 0 or 1
#finally the last reason why we do this is so that we can compare these values to the values that we expect to be in the classificaiton
true_values = np.array(y_test[0:100]) #these are the true labels
incorrect = np.where(clas_pred != true_values)
true_values.shape
incorrect
# print(incorrect[0])

(array([ 0,  0,  0, ..., 99, 99, 99]), array([ 3,  6,  7, ..., 93, 97, 98]))

In [0]:
print(len(y_test))

1000


In [0]:
idices = incorrect[0]
idices

array([ 0,  0,  0, ..., 99, 99, 99])

In [0]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [0]:
print(text[idices])
print(labels[idices])

['Kickers on my watchlist $XIDE $TRIT $SOQ $PNK $CPWR $BPZ $ALJ  trade method 1 or method 2, see prev posts'
 'Kickers on my watchlist $XIDE $TRIT $SOQ $PNK $CPWR $BPZ $ALJ  trade method 1 or method 2, see prev posts'
 'Kickers on my watchlist $XIDE $TRIT $SOQ $PNK $CPWR $BPZ $ALJ  trade method 1 or method 2, see prev posts'
 ... '$VRNG buys vs. Sells?' '$VRNG buys vs. Sells?'
 '$VRNG buys vs. Sells?']
['negative' 'negative' 'negative' ... 'positive' 'positive' 'positive']


In [0]:
# this is testing some random text to see if it is positve or negative. 
# text1 = '$APPL is definitely going to run. I expect it to gain .15c today!'
text2 = 'Sell APPL stocks now!'
textsl = [text2]
tokens = tokenizer.texts_to_sequences(textsl)
tokens_pad = pad_sequences(tokens, maxlen = max_len)
prediction = model.predict_classes(tokens_pad)
prediction

array([[0]], dtype=int32)

In [0]:
#how to get the indexes of all the true negative values
negatively = df.loc[df['sentiment'] == 'negative']
values = negatively.index.tolist()
values

[6,
 7,
 21,
 26,
 31,
 34,
 38,
 52,
 54,
 58,
 59,
 87,
 97,
 102,
 104,
 107,
 109,
 113,
 114,
 115,
 116,
 117,
 119,
 120,
 124,
 135,
 137,
 143,
 152,
 153,
 154,
 155,
 156,
 161,
 164,
 165,
 173,
 176,
 178,
 180,
 181,
 183,
 185,
 186,
 188,
 200,
 202,
 204,
 206,
 212,
 227,
 232,
 238,
 239,
 246,
 250,
 259,
 263,
 268,
 272,
 274,
 277,
 282,
 286,
 287,
 288,
 289,
 290,
 291,
 294,
 295,
 297,
 300,
 302,
 309,
 322,
 323,
 324,
 325,
 330,
 335,
 336,
 338,
 339,
 340,
 341,
 344,
 345,
 346,
 349,
 350,
 353,
 355,
 359,
 361,
 362,
 365,
 366,
 368,
 369,
 370,
 374,
 378,
 383,
 385,
 389,
 395,
 396,
 397,
 399,
 400,
 402,
 404,
 413,
 414,
 420,
 422,
 425,
 427,
 430,
 431,
 432,
 433,
 436,
 438,
 439,
 442,
 443,
 448,
 449,
 453,
 455,
 456,
 461,
 462,
 464,
 471,
 472,
 474,
 475,
 476,
 483,
 486,
 488,
 490,
 492,
 493,
 494,
 496,
 497,
 498,
 502,
 503,
 505,
 506,
 516,
 517,
 518,
 523,
 525,
 527,
 528,
 529,
 530,
 534,
 536,
 537,
 538,
 541,
 

In [0]:
#creating a new dataframe with just those negative values
negative_df = df.iloc[values]

In [0]:
negative_text = negative_df['text'].values #this returns just the negative text

In [0]:
# we will test on the first 100 negative values to see if they are classified correctly.
# this is not a valid form of testing because most of the values have already been tested and trainied on..

tokens = tokenizer.texts_to_sequences(negative_text[0:100])
tokens_pad = pad_sequences(tokens, maxlen = max_len)
prediction = model.predict_classes(tokens_pad)
print(prediction)
# false = np.where(prediction > .5)
# false

[[0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]]


In [0]:
negative_text[12]

'$AIG American International Group Option Traders bet on 4% down move by next Friday URL '

In [0]:
falsely_classified = negative_text[false[0]]
print(falsely_classified[0])

$LULU red, not ready for break out.


In [0]:
#this is testing the sentiment analysis
positive = df.loc[df['sentiment'] == 'positive']
positive_text = positive['text'].values

tokens = tokenizer.texts_to_sequences(positive_text[100:200])
tokens_pad = pad_sequences(tokens, maxlen = max_len)
pre = model.predict(tokens_pad)
#predict_classes shows the output either a 0 or 1 making it much easier to intrepret
pre = model.predict_classes(tokens_pad)
print(pre)
false = np.where(prediction > .5)
false

[[0.9848696 ]
 [0.86262435]
 [0.98121625]
 [0.98884284]
 [0.07823612]
 [0.98560405]
 [0.9827214 ]
 [0.98902285]
 [0.97623116]
 [0.9893722 ]
 [0.98872584]
 [0.9781983 ]
 [0.9881813 ]
 [0.9683452 ]
 [0.99039453]
 [0.3597659 ]
 [0.9894567 ]
 [0.9867395 ]
 [0.9754174 ]
 [0.9892541 ]
 [0.983868  ]
 [0.9845463 ]
 [0.98686975]
 [0.9881045 ]
 [0.98976773]
 [0.08143286]
 [0.99023485]
 [0.98980707]
 [0.96539026]
 [0.95243573]
 [0.08189831]
 [0.98739296]
 [0.98809475]
 [0.983214  ]
 [0.9877029 ]
 [0.9847231 ]
 [0.9881516 ]
 [0.07540542]
 [0.93922687]
 [0.98620564]
 [0.9713596 ]
 [0.9841695 ]
 [0.98925215]
 [0.980911  ]
 [0.9877436 ]
 [0.9817535 ]
 [0.09801843]
 [0.9718165 ]
 [0.97959137]
 [0.97582936]
 [0.98114544]
 [0.9845255 ]
 [0.9856894 ]
 [0.9844387 ]
 [0.98794353]
 [0.97958714]
 [0.9862389 ]
 [0.06488881]
 [0.97880965]
 [0.8945298 ]
 [0.8124025 ]
 [0.98696274]
 [0.9868318 ]
 [0.9822174 ]
 [0.9867345 ]
 [0.9871897 ]
 [0.99065524]
 [0.98490477]
 [0.9758011 ]
 [0.98862517]
 [0.9609329 ]
 [0.97

(array([18, 24, 28, 31, 34, 40, 45, 49, 52, 53, 56, 66, 67, 71, 80, 83, 97]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [0]:
falsely_classified = positive_text[false[0]]
print(len(falsely_classified)) # only 6 misclassifications in the positive test
print(falsely_classified[:])

17
['Over extended stock market + unexpected news from the FED minutes = Down $EURUSD & a panicky $ES_F $NQ_F.'
 '$BAC what happens next? URL '
 '$AAPL moving down with volume. Couldnt cope with the descending trend line. Trend is still down. Perhaps 460 is not impossible after all.'
 '$AAPL people be realistic. Market undecided what to do with apple. Earnings will lift that cloud. If GOOD we might REALLY see a reversal.'
 "$AMZN alert update: a last minute push tried to shake out puts. we're hanging on for now. URL "
 '$HMA unusual option activity. Trader buys 11k of the Jan 9 Put. Short term bear play'
 '$ZNGA Still shows red and dead on a Renko view, PF Box size 0.25 URL '
 '@user $BAC Yeah ... they made the easy profits by slashing jobs, now they have to close win-win sales.'
 '$ko has trouble staying above 200 day sma past few weeks. could short here or higher stops above 38 URL '
 '$mako a red day monday breaks support and could test old lows of 9-9.25 URL '
 'I think $AAPL is he

In [0]:
print(positive_text[:20])

['$AAPL - @user if so then the current downtrend will break. Otherwise just a short-term correction in med-term downtrend.'
 "Monday's relative weakness. $NYX $WIN $TIE $TAP $ICE $INTU $BMC $AON $CL $CHK $BIIB URL "
 "Won't believe $AAPL uptrend is back until it crosses above MA(50)"
 '$LULU red, not ready for break out.'
 '"@user: been adding $UVXY long off the bottom today for trade, also got $WPI near low"'
 '$LNKD looking like a good short. Failed to break price level resistance at 116 today.'
 'Too early to short into this move. Stock Market needs a few days to settle down. #Patience URL $COH $BWLD $DLTR $AAPL $PAY'
 '$PHM PulteGroup Option Bear bets $1.5 Million on 11% down move by April URL '
 'Short Setups displaying relative weakness: $ARO $COH $PAY $BWLD $DLTR $SHLD $WTW $UPL Prepare to short the stall. $ES_F URL '
 '$SKUL to the GRAVE!'
 '$ovti ttm ending Oct 2012 Negative $151 million operational cash flow.. 273% decline vs yoy ttm ending oct 2011'
 '$GPS wow that wa s a fa

In [0]:
model.save('stock_sentiment.h5')

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a file.
uploaded = drive.CreateFile({'title': 'Stock_sentiment.h5'})
uploaded.SetContentFile('stock_sentiment.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1kNbLFXYLrbnA0dBNftXz5cSmSAk7oRMo


In [0]:
predictions = model.predict_classes(x= X_test[:]) 
#this prints out the predictions in a more readible manner
predictions = predictions.T[0]
predictions

array([0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,

In [0]:
print(y_test[:].T)

[[0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 0 1 1 0 0 1 0 1 1
  1 1 0 0 1 0 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 0 1 1 1
  1 1 1 1 0 1 1 1 0 0 1 1 0 1 1 1 1 0 0 0 1 0 1 1 1 0 1 1 1 0 0 1 1 0 1 1
  1 1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 0 1 1
  1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0
  1 0 1 0 0 1 0 1 1 1 1 1 0 1 0 0 1 1 1 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1
  0 0 1 1 0 0 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 0 0 1 0 1 1
  1 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 0 1 0 1 0 1 1
  1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 0 0 1 1 1 0 1 1 1 1 1 1 0 0 0 1
  1 1 0 0 1 1 0 0 1 1 1 0 0 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 1 0 1 1 0 0 0 0
  1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 1 0 1 1 1 1
  1 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1
  0 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1
  0 1 0 0 0 1 0 1 0 0 0 1 0 1 1 1 1 1 

In [0]:
correct = np.array(y_test[:].T) #these are the labels associated with those test variables that we gather
incorrect= np.where(predictions != correct)
incorrect = incorrect[0]
print(len(incorrect))

249


In [0]:
'''From these tests we learned a couple things
predict_classes predicts the output without any probabilities involved
We can check which are incorrect by comparing the predicted probabilities with the actual 
labels.
The reason why the np.where clause was not executing the first time was because of the .T we transposed
the results in order to make the legible, but disregarded the fact that we would be comparing them to 
another array which would have differing dimensions.'''